In [1]:
from fddbenchmark import FDDDataset, FDDDataloader, FDDEvaluator
from fdd_defense.attacks import FGSMAttack
from fdd_defense.models.base import BaseTorchModel
from fdd_defense.models import GRU
import torch. nn as nn
from torch.optim import Adam
import torch
import torch.nn.functional as F
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from fdd_defense.defenders import DistillationDefender, QuantizationDefender, AdversarialTrainingDefender

Загружаем маленький теп

In [2]:
dataset = FDDDataset(name='small_tep')
dataset.df.head()

Reading data/small_tep/test_mask.csv: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 153300/153300 [00:00<00:00, 3938156.83it/s]


xmeas_1  xmeas_2  xmeas_3  xmeas_4  xmeas_5  xmeas_6  \
run_id    sample                                                         
413402073 1       0.25038   3674.0   4529.0   9.2320   26.889   42.402   
          2       0.25109   3659.4   4556.6   9.4264   26.721   42.576   
          3       0.25038   3660.3   4477.8   9.4426   26.875   42.070   
          4       0.24977   3661.3   4512.1   9.4776   26.758   42.063   
          5       0.29405   3679.0   4497.0   9.3381   26.889   42.650   

                  xmeas_7  xmeas_8  xmeas_9  xmeas_10  ...   xmv_2   xmv_3  \
run_id    sample                                       ...                   
413402073 1        2704.3   74.863   120.41   0.33818  ...  53.744  24.657   
          2        2705.0   75.000   120.41   0.33620  ...  53.414  24.588   
          3        2706.2   74.771   120.42   0.33563  ...  54.357  24.666   
          4        2707.2   75.224   120.39   0.33553  ...  53.946  24.725   
          5        2705.1   75.388   120.39   0.32632  ...  53.658  28.797   

                   xmv_4   xmv_5   xmv_6   xmv_7   xmv_8   xmv_9  xmv_10  \
run_id    sample                                                           
413402073 1       62.544  22.137  39.935  42.323  47.757  47.510  41.258   
          2       59.259  22.084  40.176  38.554  43.692  47.427  41.359   
          3       61.275  22.380  40.244  38.990  46.699  47.468  41.199   
          4       59.856  22.277  40.257  38.072  47.541  47.658  41.643   
          5       60.717  21.947  39.144  41.955  47.645  47.346  41.507   

                  xmv_11  
run_id    sample          
413402073 1       18.447  
          2       17.194  
          3       20.530  
          4       18.089  
          5       18.461  

[5 rows x 52 columns]

In [3]:
scaler = StandardScaler()
scaler.fit(dataset.df[dataset.train_mask])
dataset.df[:] = scaler.transform(dataset.df)

Обучаем целевую модель

In [4]:
window_size = 10

fddmodel = GRU(
        window_size=window_size,
        step_size=1,
        hidden_dim=166,
        num_epochs=28,
        batch_size=396,
        lr=0.002131,
        dropout=0.07301
    )
fddmodel.fit(dataset)

Creating sequence of samples: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 3232.59it/s]


Epochs ...:   0%|          | 0/28 [00:00<?, ?it/s]

Steps ...:   0%|          | 0/130 [00:00<?, ?it/s]

Epoch 1, Loss: 1.0932


Steps ...:   0%|          | 0/130 [00:00<?, ?it/s]

...

Epoch 26, Loss: 0.1870


Steps ...:   0%|          | 0/130 [00:00<?, ?it/s]

Epoch 27, Loss: 0.1864


Steps ...:   0%|          | 0/130 [00:00<?, ?it/s]

Epoch 28, Loss: 0.1970


Замеряем метрики на тестовой выборке

In [6]:
test_loader = FDDDataloader(
    dataframe=dataset.df,
    mask=dataset.test_mask,
    labels=dataset.labels,
    window_size=window_size,
    step_size=1,
)
for sample, index, label in test_loader:
    pred = fddmodel.predict(sample)
pred = pd.Series(pred, index=index, name='pred')

evaluator = FDDEvaluator(step_size=1)
evaluator.print_metrics(label, pred)

Creating sequence of samples: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 3192.91it/s]


FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9905/0.0000
    Fault 02: 0.9855/0.0000
    Fault 03: 0.0000/0.0000
    Fault 04: 0.9882/0.0000
    Fault 05: 0.9937/0.0000
    Fault 06: 0.7369/0.0000
    Fault 07: 0.9985/0.0000
    Fault 08: 0.8531/0.0000
    Fault 09: 0.0000/0.0000
    Fault 10: 0.8773/0.0117
    Fault 11: 0.9660/0.0000
    Fault 12: 0.7036/0.0000
    Fault 13: 0.9086/0.0039
    Fault 14: 0.9950/0.0000
    Fault 15: 0.0000/0.0000
    Fault 16: 0.9041/0.0050
    Fault 17: 0.9442/0.0000
    Fault 18: 0.7657/0.0000
    Fault 19: 0.9622/0.0222
    Fault 20: 0.8761/0.0048
Detection TPR: 0.8287
Detection FPR: 0.0477
Average Detection Delay (ADD): 22.90
Total Correct Diagnosis Rate (Total CDR): 0.9321

Clustering metrics
-----------------
Adjusted Rand Index (ARI): 0.5778
Normalized Mutual Information (NMI): 0.8220
Unsupervised Clustering Accuracy (ACC): 0.8083


В результате имеет правильную диагностику в 87% а так же FP в 34%, но это результат использования маленького датасета

In [11]:
defender = QuantizationDefender(fddmodel, qbit=10)
attack = FGSMAttack(model=defender.fddmodel, eps=0.05)
for sample, index, label in test_loader:
    _sample = torch.FloatTensor(sample)
    _label = torch.LongTensor(label.values)
adv_sample = attack.attack(_sample, _label)

In [12]:
pred = defender.predict(adv_sample)
pred = pd.Series(pred, index=index, name='pred')

evaluator = FDDEvaluator(step_size=1)
evaluator.print_metrics(label, pred)

FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9735/0.0000
    Fault 02: 0.9810/0.0004
    Fault 03: 0.0000/0.0000
    Fault 04: 0.5447/0.0001
    Fault 05: 0.9106/0.0012
    Fault 06: 0.7547/0.0000
    Fault 07: 0.9975/0.0000
    Fault 08: 0.6333/0.0151
    Fault 09: 0.0000/0.0000
    Fault 10: 0.4874/0.0994
    Fault 11: 0.8616/0.0256
    Fault 12: 0.6541/0.0002
    Fault 13: 0.7572/0.0282
    Fault 14: 0.9867/0.0000
    Fault 15: 0.0000/0.0000
    Fault 16: 0.6085/0.0420
    Fault 17: 0.9096/0.0278
    Fault 18: 0.4969/0.0051
    Fault 19: 0.8541/0.0771
    Fault 20: 0.6123/0.0877
Detection TPR: 0.8447
Detection FPR: 0.4099
Average Detection Delay (ADD): 8.57
Total Correct Diagnosis Rate (Total CDR): 0.7709

Clustering metrics
-----------------
Adjusted Rand Index (ARI): 0.3705
Normalized Mutual Information (NMI): 0.6201
Unsupervised Clustering Accuracy (ACC): 0.6390


TPR 84% FPR 40% CDR 77%